# Rest API

Zistenie dát z rest API:
- vhodné je skontrolovať status odpovede
- ak nie je 200, vrátiť chybu

In [2]:
import requests

requests.get odošle HTTP GET požiadavku na zadnú URL a získa odpoveď od servera

Odpoveď obsahuje:
Stavový kód (status_code): výsledok požiadavky (napr. 200 je OK, 404 fail ..)
Hlavičky (headers): Metadáta o výsledku: typ obsahu, kódovanie
Obsah (content): Samotné dáta, ktoré server vrátil.

In [3]:
response = requests.get('http://covid-api.com/api/reports')
response.__class__

requests.models.Response

In [4]:
# stavový kód odpovede
response.status_code

200

In [5]:
# hlavička odpovede
response.headers

{'Server': 'nginx/1.14.0 (Ubuntu)', 'Content-Type': 'application/json', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'Cache-Control': 'no-cache, private', 'Date': 'Wed, 21 Aug 2024 15:17:42 GMT', 'X-RateLimit-Limit': '2000', 'X-RateLimit-Remaining': '1999'}

In [6]:
# prvých 200 znakov
response.text[:200]

'{"data":[{"date":"2023-03-09","confirmed":209451,"deaths":7896,"recovered":0,"confirmed_diff":0,"deaths_diff":0,"recovered_diff":0,"last_update":"2023-03-10 04:21:03","active":201555,"active_diff":0,"'

response.json(): Pokúsi sa dekódovať obsah odpovede ako JSON a vráti ho ako slovník alebo zoznam v Pythone

In [7]:
response_json = response.json()

In [8]:
response_json.__class__
# response je slovník

dict

## Úloha: Načítaj cez API JSON formát 

Vytvor funkciu, ktorá pre zadané url a endpoint vráti odpoveď v JSON formáte. Ak je HTTP odpoveď iná ako 200, vypíše chybové hlásenie

In [9]:
def rest_data(url='http://covid-api.com/api', endpoint='/reports'):
    response = requests.get(url+endpoint)
    if response.status_code == 200: return response.json()
    else:
        raise ValueError('Chyba: ', response.status_code)

In [10]:
data_api = rest_data()

In [11]:
data_api.__class__

dict

In [12]:
len(data_api)

1

In [13]:
data_api.keys()

dict_keys(['data'])

dáta načítané z api sú v premennej typu dictionary, kde jediným kľúčom je reťazec 'data'

In [14]:
# dáta sú prvý prvok v dictionary
data = data_api['data']

In [15]:
data.__class__

list

In [16]:
len(data)

791

dáta sú uložené v zozname, ktorý obsahuje 791 záznamov

In [17]:
# prvý záznam
data[0].__class__

dict

každý záznam v liste je dictionary

In [19]:
data[0].keys()

dict_keys(['date', 'confirmed', 'deaths', 'recovered', 'confirmed_diff', 'deaths_diff', 'recovered_diff', 'last_update', 'active', 'active_diff', 'fatality_rate', 'region'])

In [21]:
from pprint import pprint

In [22]:
# "pekne" vypíš prvý záznam
pprint(data[0])

{'active': 201555,
 'active_diff': 0,
 'confirmed': 209451,
 'confirmed_diff': 0,
 'date': '2023-03-09',
 'deaths': 7896,
 'deaths_diff': 0,
 'fatality_rate': 0.0377,
 'last_update': '2023-03-10 04:21:03',
 'recovered': 0,
 'recovered_diff': 0,
 'region': {'cities': [],
            'iso': 'AFG',
            'lat': '33.9391',
            'long': '67.7100',
            'name': 'Afghanistan',
            'province': ''}}


## Konverzia do pandas.Dataframe a normalizácia stĺpcov

In [23]:
import pandas as pd

In [24]:
df = pd.json_normalize(data, sep='_')
df

,date,confirmed,deaths,recovered,confirmed_diff,deaths_diff,recovered_diff,last_update,active,active_diff,fatality_rate,region_iso,region_name,region_province,region_lat,region_long,region_cities
0,2023-03-09,209451,7896,0,0,0,0,2023-03-10 04:21:03,201555,0,0.0377,AFG,Afghanistan,,33.9391,67.7100,[]
1,2023-03-09,334457,3598,0,14,0,0,2023-03-10 04:21:03,330859,14,0.0108,ALB,Albania,,41.1533,20.1683,[]
2,2023-03-09,271496,6881,0,2,0,0,2023-03-10 04:21:03,264615,2,0.0253,DZA,Algeria,,28.0339,1.6596,[]
3,2023-03-09,47890,165,0,0,0,0,2023-03-10 04:21:03,47725,0,0.0034,AND,Andorra,,42.5063,1.5218,[]
4,2023-03-09,105288,1933,0,0,0,0,2023-03-10 04:21:03,103355,0,0.0184,AGO,Angola,,-11.2027,17.8739,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
786,2023-03-09,11526994,43186,0,0,0,0,2023-03-10 04:21:03,11483808,0,0.0037,VNM,Vietnam,,14.0583,108.2772,[]
787,2023-03-09,703228,5708,0,0,0,0,2023-03-10 04:21:03,697520,0,0.0081,PSE,West Bank and Gaza,,31.9522,35.2332,[]
788,2023-03-09,11945,2159,0,0,0,0,2023-03-10 04:21:03,9786,0,0.1807,YEM,Yemen,,15.552726999999999,48.516388,[]
789,2023-03-09,343135,4057,0,0,0,0,2023-03-10 04:21:03,339078,0,0.0118,ZMB,Zambia,,-13.1339,27.8493,[]


## Výber údajov jednej krajiny z načítaných dát

In [26]:
pprint(data[0])

{'active': 201555,
 'active_diff': 0,
 'confirmed': 209451,
 'confirmed_diff': 0,
 'date': '2023-03-09',
 'deaths': 7896,
 'deaths_diff': 0,
 'fatality_rate': 0.0377,
 'last_update': '2023-03-10 04:21:03',
 'recovered': 0,
 'recovered_diff': 0,
 'region': {'cities': [],
            'iso': 'AFG',
            'lat': '33.9391',
            'long': '67.7100',
            'name': 'Afghanistan',
            'province': ''}}


country uložená vo vnorenom dictionary, čiže kľúčom sú postupne: region, name

### Cez for cyklus

In [28]:
for record in data:
    if record['region']['name'] == 'Slovakia': 
        print(record.__class__)
        print(record)

<class 'dict'>
{'date': '2023-03-09', 'confirmed': 2667551, 'deaths': 21035, 'recovered': 0, 'confirmed_diff': 262, 'deaths_diff': 2, 'recovered_diff': 0, 'last_update': '2023-03-10 04:21:03', 'active': 2646516, 'active_diff': 260, 'fatality_rate': 0.0079, 'region': {'iso': 'SVK', 'name': 'Slovakia', 'province': '', 'lat': '48.6690', 'long': '19.6990', 'cities': []}}


### Boolean indexing (maska)

In [29]:
df = pd.json_normalize(data_api['data'], sep='_')
df[df.region_name=='Slovakia']

,date,confirmed,deaths,recovered,confirmed_diff,deaths_diff,recovered_diff,last_update,active,active_diff,fatality_rate,region_iso,region_name,region_province,region_lat,region_long,region_cities
611,2023-03-09,2667551,21035,0,262,2,0,2023-03-10 04:21:03,2646516,260,0.0079,SVK,Slovakia,,48.6690,19.6990,[]


In [30]:
df.__class__

pandas.core.frame.DataFrame

### Pomocou metódy query()

In [31]:
df.query("region_name == 'Slovakia'")

,date,confirmed,deaths,recovered,confirmed_diff,deaths_diff,recovered_diff,last_update,active,active_diff,fatality_rate,region_iso,region_name,region_province,region_lat,region_long,region_cities
611,2023-03-09,2667551,21035,0,262,2,0,2023-03-10 04:21:03,2646516,260,0.0079,SVK,Slovakia,,48.6690,19.6990,[]
